# How to Specify a Model

One of the core features of **respy** is its flexible modeling capabilities. The guide on *example models* showcases a collection of economic models, that have already been implemented and can be accessed freely. 

However, the package can also be used to implement models from scratch. This guide illustrates how to translate an economic model from mathematical equations to the `params` and `options` which **respy** uses to implement a model. To to so, we will draw on the seminal work of Keane and Wolpin (1994) to replicate their dynamic discrete choice model of schooling and occupational choice.

---
    
Note that your model has to fall into the class of so called Eckstein-Keane-Wolpin (EKW) models in order to be implementable in **respy**. You can find further information about this modeling framework in the explanations section of this documentation.
       

---

## Economic model

The model from Keane and Wolpin (1994) is a close relative of the models implemented in Keane and Wolpin (1997). Refer to the explanations section of this documentation for a detailed outline of the economic model and mathematical model. The model outline below builds on these explanations and is thus not exhaustive of all model components on its own.

In short, we are working with a dynamic choice model which follows individuals as they decide between working and schooling at different points in their life-cycle. They accumulate experience based on their choices and are forward-looking individuals which take the value of future outcomes (which they discount by a factor $\delta$) into account when making choices in the present. Simultaneously, all choices are subject to alternative-specific shocks which creates uncertainty about the future in the individuals' choice problem. The model of Keane and Wolpin (1994) constitutes a simplified version of the model discussed in the explanations. We can see the differences when assessing the reward and choice structure of the model.

The model of Keane and Wolpin (1994), which we use for illustrative purposes in this guide, includes $a \in \{1,2,3,4\}$ mutually exclusive choice alternatives: working in occupation *A* or *B* ($a=1,2$), investing in *education* ($a=3$), or staying *home* ($a=4$). The rewards for these four choice alternatives in each of the 40 periods $t \in \{0, ...,39\}$ are specified below:

$$
\begin{align}
    \label{sec:rewardfunc}
    \text{Occupation A: } R_1(t) & = w_{1t} = r_{1}exp\{\alpha_{10} + \alpha_{11}s_{t} + \alpha_{12}x_{1t} + \alpha_{13}x^2_{1t} + \alpha_{14}x_{2t} + \alpha_{15}x^2_{2t} + \epsilon_{1t}\} \nonumber\\
    \text{Occupation B: } R_2(t) & = w_{2t} = r_{2}exp\{\alpha_{20} + \alpha_{21}s_{t} + \alpha_{22}x_{1t} + \alpha_{23}x^2_{1t} + \alpha_{24}x_{2t} + \alpha_{25}x^2_{2t} + \epsilon_{2t}\} \nonumber\\
    \text{School: }R_3(t) & = \beta_0 + \beta_{1}I(s_t \geq 12) + \beta_2(1-d_3(t-1)) + \epsilon_{3t}, \nonumber\\
    \text{Home: }R_4(t) & = \gamma_0 + \epsilon_{4t},
\end{align}
$$

These rewards enter the alternative specific value functions of individuals and include the parameters structural estimation seeks to identify. In these equations $s(t)$ denotes schooling in period $t$ and $x_{at}$ denotes work experience from sector $A$ or $B$ ($a=1,2$). The reward for schooling includes an indicator $I(s_t \geq 12)$ which is connected to the cost of schooling after 12 periods (i.e. post-secondary schooling costs) and component that captures costs of returning to school when the choice in the previous period was something else. Aside from the parameters connected to these various components, each reward function also contains a constant and an alternative specific shock. The skill price in occupations is denoted by $r_{a}$, it is set to 1 in this model. Note that the reward functions are not only time but also individual specific. A subscript for individuals is left out for simplicity.

## Components to modeling

How can we map these equations into **respy** to construct a discrete choice dynamic programming model that allows us to estimate the structural parameters? 

A model in **respy** is defined by two components:

1. The `params` DataFrame, where model parameters reside. It should be specified as a [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html). 

2. The `options` which specify the settings for the model solution and further restrictions on the model structure. `options` are defined in a Python `dictionary`. Examples of components that enter the options include the number of periods, type of numerical integration, unfeasible states, etc. 

In the next steps, we will examine these two components in detail to illustrate how they mirror the model outlined above. Since the model of Keane and Wolpin (1994) is already implemented, we can simply load it into memory.

In [1]:
import respy as rp

In [2]:
params, options, data = rp.get_example_model("kw_94_one")

---

Note that when you specify these objects yourself, doing so in separate files might facilitate your workflow. For example, `params` could be loaded from a .csv-file and `options` from a .yaml-file.

---

## Specifying the `params`

We first inspect the `params` DataFrame. It contains all the parameters that enter the structural model. Usually, these parameters will be estimable, but this is not mandatory. For instance, a specified shock distribution may guide the model but exogenously set. The `params` DataFrame may also contain auxiliary parameters that aid simulation but do not stem from the value functions of the different choice alternatives. **respy** allows copious freedom in designing reward functions and naming parameters. However, certain rules need to be accounted for to allow **respy** to process a model correctly. Below, we discuss each parameter group of our exemplary `params` DataFrame to outline how parameters can be specified.

In [3]:
params

value  \
category            name                                  
delta               delta                        0.9500   
wage_a              constant                     9.2100   
                    exp_edu                      0.0380   
                    exp_a                        0.0330   
                    exp_a_square                -0.0005   
                    exp_b                        0.0000   
                    exp_b_square                 0.0000   
wage_b              constant                     8.4800   
                    exp_edu                      0.0700   
                    exp_b                        0.0670   
                    exp_b_square                -0.0010   
                    exp_a                        0.0220   
                    exp_a_square                -0.0005   
nonpec_edu          constant                     0.0000   
                    at_least_twelve_exp_edu      0.0000   
                    not_edu_last_period      -4000.0000   
nonpec_home         constant                 17750.0000   
shocks_sdcorr       sd_a                         0.2000   
                    sd_b                         0.2500   
                    sd_edu                    1500.0000   
                    sd_home                   1500.0000   
                    corr_b_a                     0.0000   
                    corr_edu_a                   0.0000   
                    corr_edu_b                   0.0000   
                    corr_home_a                  0.0000   
                    corr_home_b                  0.0000   
                    corr_home_edu                0.0000   
lagged_choice_1_edu probability                  1.0000   
initial_exp_edu_10  probability                  1.0000   
maximum_exp         edu                         20.0000   

                                                                                       comment  
category            name                                                                        
delta               delta                                                      discount factor  
wage_a              constant                                               log of rental price  
                    exp_edu                          return to an additional year of schooling  
                    exp_a                                     return to same sector experience  
                    exp_a_square                   return to same sector, quadratic experience  
                    exp_b                                    return to other sector experience  
                    exp_b_square                  return to other sector, quadratic experience  
wage_b              constant                                               log of rental price  
                    exp_edu                          return to an additional year of schooling  
                    exp_b                                     return to same sector experience  
                    exp_b_square                   return to same sector, quadratic experience  
                    exp_a                                    return to other sector experience  
                    exp_a_square                  return to other sector, quadratic experience  
nonpec_edu          constant                            constant reward for choosing education  
                    at_least_twelve_exp_edu        reward for going to college (tuition, etc.)  
                    not_edu_last_period                        reward for going back to school  
nonpec_home         constant                         constant reward of non-market alternative  
shocks_sdcorr       sd_a                     Element 1,1 of standard-deviation/correlation ...  
                    sd_b                     Element 2,2 of standard-deviation/correlation ...  
                    sd_edu                   Element 3,3 of standard-deviation/correlation ...  
                    sd_home

### Index stucture

The `params` DataFrame needs to abide to a specific index structure:

- **Index**: The DataFrame has a MultiIndex with two levels. The levels have to be named `category` and `name`. Categories need to be unique. Names may be repeated but never within the same category. This ensures that each parameter is uniquely identfied in the `params` DataFrame.
- **Columns**:The parameter value needs to be saved in a column called `value`. The `params` DataFrame may contain other columns like the comment column above. They do not influence the model. This can also be useful for parameter estimation where information like bounds may need to be specified as additional columns.

### Discounting

In `respy` the discount factor has a pre-defined and unmutable name: `delta`.

In [4]:
params.loc["delta"]

,value,comment
name,,
delta,0.95,discount factor


**respy** also supports hyperbolic discounting. You can implement it in your model by adding a `category` and `name` called `beta` to your parameter vector.

### Choice Rewards

The structural model consists of two building blocks: states and choices. Choices in general can have two types of rewards: 

- **pecuniary rewards**, i.e. wages, with corresponding `category`: `wage_{choice}`
- **non-pecuniary rewards**, i.e. intrinsic value of education, , with corresponding `category`: `nonpec_{choice}`

Choices can be named freely but it is important to use the appropriate prefixes so **respy** can process the model accordingly. In our example above, choices have either exclusively pecuniary rewards (occupation *A* and *B*) or non-pecuniary rewards (*education* and *home*) but **respy** also allows for combinations of both types to define reward functions. Each parameter in `params` then corresponds to a parameter in the reward functions.

#### Example: Returns to Occupation A

Take for example the reward function for choosing to work in occupation *A*:

$$
R_1(t) = w_{1t} = r_{1} exp\{\alpha_{10} + \alpha_{11}s_{t} + \alpha_{12}x_{1t} - \alpha_{13}x^2_{1t} + \alpha_{14}x_{2t} - \alpha_{15}x^2_{2t} + \epsilon_{1t}\} \nonumber\\
$$

We can directly map the `params` DataFrame to the equation. All parameters are saved under the `category` of `wage_a`. The pecuniary reward associated with working in occupation A, `wage_a` is determined by state-specific returns. The index `name` collects all covariates where `value` captures the associated return. The state-variables and returns are mapped to the entries in `category` `wage_a` according to the following table:

|    Covariate  |    `name`      |    Return      |  `value`  |
|---------------|----------------|----------------|-----------|
|      $1$      |   `constant`   | $\alpha_{10}$  |  $9.2100$ |
|    $s_{t}$    |    `exp_edu`   | $\alpha_{11}$  |  $0.0380$ |
|  $x_{1t}$     |     `exp_a`    | $\alpha_{12}$  |  $0.0330$ |
| $x_{1t}^2$    | `exp_a_square` | $\alpha_{13}$  | $-0.0005$ |
|  $x_{2t}$     |     `exp_b`    | $\alpha_{14}$  |  $0.0000$ |
| $x_{2t}^2$    | `exp_b_square` | $\alpha_{15}$  |  $0.0000$ |

We can imagine the equation to be written as

$$
 w_{1t} = 1 \cdot exp\{9.2100 \cdot 1 + 0.0380 \cdot s_{t} + 0.0330 \cdot x_{1t} -0.0005 \cdot x_{1t}^2 + 0.0000 \cdot x_{2t} + 0.0000 \cdot x_{2t}^2\ + \epsilon_{1t}\}.
$$



The choice-specific shock that is also part of this equation will be discussed in more detail below.

---

Note that the prefix `exp_` is a special `name` in **respy** and must be followed by the name of a choice. Parameters with this prefix indicate the return to experience in a certain choice alternative. The names `constant` and `exp_{choice}_square` conversely do not hold such special significance and require further user input in the `options` to be correctly specfied. 

Experience accumulation is a central component of EKW models and thus an important feature of **respy**. You will noice that `exp_home` appears nowhere in our DataFrame. As can be inferred from our model equations, individuals do not accumulate experience at home. Not inculding a parameter for experience at home signals to **respy** that this is not a component in our model. Choices with a wage automatically account for experience accumulation.

---

### Shocks

For each choice reward, idiosyncratic and serially uncorrelated shocks alter the respective return. Those alternative-specific shocks are specified jointly in `category` `shocks_sdcorr`.

In [5]:
params.loc["shocks_sdcorr"]

,value,comment
name,,
sd_a,0.20,"Element 1,1 of standard-deviation/correlation ..."
sd_b,0.25,"Element 2,2 of standard-deviation/correlation ..."
sd_edu,1500.00,"Element 3,3 of standard-deviation/correlation ..."
sd_home,1500.00,"Element 4,4 of standard-deviation/correlation ..."
corr_b_a,0.00,"Element 2,1 of standard-deviation/correlation ..."
corr_edu_a,0.00,"Element 3,1 of standard-deviation/correlation ..."
corr_edu_b,0.00,"Element 3,2 of standard-deviation/correlation ..."
corr_home_a,0.00,"Element 4,1 of standard-deviation/correlation ..."
corr_home_b,0.00,"Element 4,2 of standard-deviation/correlation ..."


Shocks are **assumed to follow a mutivariate normal distribution** with zero mean and covariance matrix $\Sigma$. The **dimensionality** of the symmetric covariance matrix equals the number of modeled choices. The specification of $\Sigma$ remains in the discretion of the user. Because the symmetry of covariance matrices, it is sufficient to specify the lower triangular matrix. However, it is mandatory to follow the order which is prescribed by `respy`. 

- First, the **diagonal elements (standard deviations)** are specified via `sd_{choice}` according to the following order:

    1. Working alternatives (alphabetically sorted).
    2. Non-working alternatives with experience accumulation (alphabetically sorted).
    3. Remaining alternatives (alphabetically sorted.)

- Second, the **off-diagonal elements (correlations)** are specified ordered **by rows** in the matrix. 


Aside from specifying shocks according to standard deviations and correlations, you can also specify the variance-covariance matrix. The parameters are ordered by appearance in the lower triangular. Variances have the name `var_{choice}` and covariances `cov_{choice_2}_{choice_1}` and so forth. Lastly, another option is the Cholesky factor of the variance-covariance matrix ordered by appearance in the lower triangular. The labels are either `chol_{choice}` or `chol_{choice_2}_{choice_1}` and so forth. In contrast to the other two options, Cholesky shocks are not ordered according to diagonal and off-diagonal elements. Instead they need to be ordered according to appearance by rows in the lower triangular of the shock matrix.

---
  
Specifying shocks a first glance might be confusing due to the ordering requirements. Note that **respy** raises an error if incorrectly ordered shock parameters are passed to it. The error message will help you specify the parameters in the correct order.


---



### Additional Parameters

Aside from discounting and parameters that affect the reward functions (pecunitary rewards, non-pecuniary rewards, and shocks) there are some additional  parameters that you might want to add to specify your model. Below you find a small overview of the type of parameters you may add.

#### Initial Conditions

In many instances, you may need to add initial conditions to your model. This can include lagged choices, experience levels, and observable characteristics. These characteristics are assigned probabilities with which they occur. Importantly, they are usually not estimable parameters. Our example model requires two such parameter specifications. 

The parameter `lagged_choice_1_edu` ensures that the model logs *education* as the previous choice in period $t=-1$ for all individuals in the sample. Our model requires this specification because we included a cost of returning to school in the reward function for education, if the previous choice was another alternative. In order to compute the rewards for period $0$, **respy** thus needs to know the choice of for the previous period, even if it is not directly part of the model's decision horizon. 

In [6]:
params.loc["lagged_choice_1_edu"]

,value,comment
name,,
probability,1.0,Probability that the first lagged choice is ed...


The parameter `initial_exp_edu_10` assigns 10 periods of experience in education (i.e. 10 years of schooling) to all individuals in period $0$. Adding an initial condition like this may be useful if we think about the correspondence between model and potential empirical data. Since we are assessing occupational decisions, we will be analyzing individuals of working age who will usually have accumulated schooling before they enter the labor market.

In [7]:
params.loc["initial_exp_edu_10"]

,value,comment
name,,
probability,1.0,Probability that the initial level of educatio...


Both of these parameters only exhibit one value that occurs for all individuals in this example. However, initial conditions are much more versatile and can be defined quite flexibility. Refer to the guide linked below for more information. 

#### Maximum Experience

Much like adding initial experience, we may want to limit the amount of experience an individual can accumulate over the life-cycle. In our example, educational experience is limited to 20 years. The implementation is straight forward. We define a `category` called `maximum_exp` and add a parameter `name` that corresponds to the name of a choice (e.g. `edu`). The `value` column holds the number of periods that constitute the limit.

In [8]:
params.loc["maximum_exp"]

,value,comment
name,,
edu,20.0,Maximum level of experience for education (opt...


#### Unobserved Heterogeneity

A component not implemented in this example is unobserved heterogeneity between individuals. **respy** allows to add such components using finite mixture approaches. Check out the guide below and example models based on Keane and Wolpin (1997) to learn more about adding unobserved heterogeneity to your model.

#### Measurement Error

You may also implement measurement error in wages. To do so you have to define a `category` called `meas_error` and add the parameter names `sd_{choice}` for all choices with a wage. The parameter `value` should be the standard deviations of measurement error. Check out the model parametrization of `kw_97_extended` for an example.

## Defining the `options`

The `options` dictionary is the second necessary component for defining models in **respy**. As we have learned above, structural parameters are defined in a pandas.DataFrame. The `options` dictionary holds additional settings and information about the model. However, the `params` DataFrame and `options` dictionary are not to be viewed as completely separate objects as some types of parameters require additional options in order for **respy** to process them. Below we will inspect our example model's `options` to learn more about this component of modeling.

In [9]:
options

{'estimation_draws': 200,
 'estimation_seed': 500,
 'estimation_tau': 500,
 'interpolation_points': -1,
 'n_periods': 40,
 'simulation_agents': 1000,
 'simulation_seed': 132,
 'solution_draws': 500,
 'solution_seed': 15,
 'monte_carlo_sequence': 'random',
 'core_state_space_filters': ["period > 0 and exp_{choices_w_exp} == period and lagged_choice_1 != '{choices_w_exp}'",
  "period > 0 and exp_a + exp_b + exp_edu == period and lagged_choice_1 == '{choices_wo_exp}'",
  "period > 0 and lagged_choice_1 == 'edu' and exp_edu == 0",
  "lagged_choice_1 == '{choices_w_wage}' and exp_{choices_w_wage} == 0",
  "period == 0 and lagged_choice_1 == '{choices_w_wage}'"],
 'covariates': {'constant': '1',
  'exp_a_square': 'exp_a ** 2',
  'exp_b_square': 'exp_b ** 2',
  'at_least_twelve_exp_edu': 'exp_edu >= 12',
  'not_edu_last_period': "lagged_choice_1 != 'edu'"}}

### `n_periods`

Individuals will take the actions which maximize their expected utility over the time horizon or lifetime defined by `n_periods`. Possible values are one and higher integers. This option is mandatory as no default is supplied. In most models, the model's complexity or the number of states in the state space is exponentially increasing in the number of periods.

Do not confuse this option with the number of periods for which you want to simulate the actions of individuals. This number can be lower because although actions of individuals are simulated for, say, 10 periods, their actions can still aim to maximize utility for 50 periods.

In [10]:
options["n_periods"]

40

### `simulation_agents`

This option specifies the number of individuals which are simulated. This option is ignored if you pass data to the simulation function.

In [11]:
options["simulation_agents"]

1000

### `covariates`

In the subsection on the [parameterization of the choice rewards](#Choice-Rewards), we discussed the special role of `exp_{choice}` in defining parameters for the pecuniary reward of occupation A. However, the parameter vector includes further covariates like a constant and squared experience terms. 

These covariates need further specfication so **respy** knows how to process them. Covariates with no pre-defined naming convention are specfied in the model `options` as a nested dictionary called `covariates`. In the `covariates` dictionary, keys correspond to the paramater `name` in `params` and dictionary values hold the definition of this parameter.

For example, all parameters named `constant` return a value of 1 for every individual. The parameters `exp_a_square` and `exp_b_square`  signal the return to square experience in both occupations.

The other two covariates enter the reward function for *education*. `at_least_twelve_exp_edu` is a boolean that activates when an indiviudal has accumulated 12 years of schooling or more. This triggers a cost component in the reward function when the condition evaluates to True. Lastly, the covariate `not_edu_last_period` is a boolean indicator for not having chosen *education* in the last period. As discussed in the section on [initial conditions](#Initial-Conditions), this necessitates the inclusion of a lagged choice in the parameter DataFrame.

In [12]:
options["covariates"]

{'constant': '1',
 'exp_a_square': 'exp_a ** 2',
 'exp_b_square': 'exp_b ** 2',
 'at_least_twelve_exp_edu': 'exp_edu >= 12',
 'not_edu_last_period': "lagged_choice_1 != 'edu'"}

---
    
How should covariate definitions in the `options` look like to be processed? Here are some pointers:

- The statements are evaluated using [pandas.eval](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.eval.html). This means you can use all arithmetic operations that this function supports.
- The following pre-defined terms are recognized to construct covariates: `period`, `exp_{choice}`, `lagged_choice_{number of periods}`.
- You can also refer to other covariates you have defined to construct new ones.

---

### Seeds (optional)

To be able to replicate a model, there are three possible seeds in the `options` for solution, simulation, and estimation. The distinction allows to vary randomness in only one component, independent from the others. The keys are

- `solution_seed` for the computation of the decision rules.
- `simulation_seed` for the simulation.
- `estimation_seed` for the computation of the log likelihood.

In [13]:
{k: v for k, v in options.items() if "seed" in k}

{'estimation_seed': 500, 'simulation_seed': 132, 'solution_seed': 15}

### `monte_carlo_sequence` and draws (optional)

`monte_carlo_sequence` and draws refer more generally to approximations of integrals with Monte Carlo simulations inside **respy**. There exist two applications for Monte Carlo simulation.

1. In the solution of a model, the value of expected value functions has to be simulated.
2. While computing the log likelihood, (log) choice probabilities are simulated.

The number of draws controls how many points are used to evaluate an integral. The default is 500 for the solution and 200 for the computation of choice probabilities.

In [14]:
{k: v for k, v in options.items() if "draws" in k}

{'estimation_draws': 200, 'solution_draws': 500}

The option `monte_carlo_sequence` controls how points are drawn.

- With `"random"` points are drawn randomly which is also called crude Monte Carlo simulation.
- Quasi-random Monte Carlo simulation uses low-discrepancy sequences like Sobol and Halton to draw points which require less points than random for the same approximation error. Use either `"sobol"` or `"halton"`.

### `interpolation_points`

The number of interpolation points specifies the number states or their corresponding expected value functions which are used to fit an interpolation model. The model is used to predict the expected value functions for all remaining states. The interpolation method available in **respy** is designed in Keane and Wolpin (1994).

If `interpolation_points` is set to -1, the full solution is computed.

In [15]:
options["interpolation_points"]

-1

### `negative_choice_set`

You can limit the set of available choices at different points in time using the option `negative_choice_set`. To implement a negative choice set, define a nested dictionary where keys correspond to choice alternatives and values hold a list of conditions that will eliminate the corresponding choice for periods where it evaluates to True.

Let's for example think consider a scenario where individuals could only work in occupation A after the fifth period ($t=4$) in the model (i.e. the occupation may have an age requirement). In this case, we could implement a negative choice set for these first five periods as follows.

In [16]:
options["negative_choice_set"] = {"a" : ["period < 5"]}

### `core_state_space_filters` (optional)

Core state space filters partly complement the `negative_choice_set` option. First of all, what is the core state space? The core state space is the part of the state space spanned by the combinations of experiences and previous choices. Not all combinations are possible, but it is not always possible to catch all invalid combinations.

States with impossible combinations have no effect on the correctness of the model, but pose an additional computational burden which should be eliminated. Similar to `negative_choice_set` the `core_state_space_filters` are a list of conditions and whenever one of them is true, the state is eliminated from the state space.

In [17]:
options["core_state_space_filters"]

["period > 0 and exp_{choices_w_exp} == period and lagged_choice_1 != '{choices_w_exp}'",
 "period > 0 and exp_a + exp_b + exp_edu == period and lagged_choice_1 == '{choices_wo_exp}'",
 "period > 0 and lagged_choice_1 == 'edu' and exp_edu == 0",
 "lagged_choice_1 == '{choices_w_wage}' and exp_{choices_w_wage} == 0",
 "period == 0 and lagged_choice_1 == '{choices_w_wage}'"]

--- 

Core state space filters are applied **before** initial conditions are implemented. Pay attention to this when you have, for example, implemented initial experience for a choice. When adding a filter based on the experience for this choice, you will have to refer to within-model experience and discard knowledge of potential previous experience. 

Negative choice sets on the other hand are applied **after** initial conditions are implemented.

---

### `estimation_tau`

*This option is relevant for maximum likelihood estimation.*

The tau parameter is also called the temperature parameter of a [softmax function](https://en.wikipedia.org/wiki/Softmax_function) which is used to compute the choice probabilities in the maximum likelihood estimation. For $\tau \to \infty$ all choices become equiprobable whereas for $\tau \to 0$ some choices receive a zero probability which is not desirable while using gradient-based numerical optimization methods.

The parameter has a huge impact on the log likelihood of a sample and seems to be model dependent. In Keane and Wolpin (1994) and related literature, the parameter is set to 500. We recommend to test different values ranging from >0 to 500. Lower values are only possible because respy computes the log likelihood solely in the log-space and uses robust methods to avoid under- and overflows.

In [18]:
options["estimation_tau"]

500



## References

- Keane, M. P., & Wolpin, K. I. (1994). The Solution and Estimation of Discrete Choice Dynamic Programming Models by Simulation and Interpolation: Monte Carlo Evidence. *The Review of Economics and Statistics*, 648-672.

- Keane, M. P., & Wolpin, K. I. (1997). The Career Decisions of Young Men. *Journal of Political Economy*, 105(3), 473-522.